In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [17]:
df = pd.read_csv('./data/training_data/stock_metrics.csv') #
df['greater_than'] = df['price_change'].apply(lambda x: 1 if x > 1.2 else 0)
print(df.shape)
df.head()

(14304, 20)


,ticker,purchase_volume,purchase_speculation,purchase_count,purchase_count_individual,purchase_days_ago,purchase_owner,purchase_confidence,sale_volume,sale_speculation,sale_count,sale_count_individual,sale_days_ago,sale_owner,sale_confidence,date,volume_net,score,price_change,greater_than
0,DD,2.0,0,2,1,62.5,['Thomas R Carper'],0.261,0.0,0,0,0,-1.0,[],0.0,2013-01-01,2.0,1.04,1.42,1
1,DD,1.0,0,1,1,75.0,['Thomas R Carper'],0.261,0.0,0,0,0,-1.0,[],0.0,2013-03-02,1.0,0.52,1.59,1
2,DD,1.0,0,1,1,47.0,['Thomas R Carper'],0.414,0.0,0,0,0,-1.0,[],0.0,2013-05-01,1.0,0.83,1.52,1
3,DD,2.0,0,2,1,62.0,['Thomas R Carper'],0.572,0.0,0,0,0,-1.0,[],0.0,2013-06-30,2.0,2.29,1.65,1
4,DD,1.0,0,1,1,77.0,['Thomas R Carper'],0.572,0.0,0,0,0,-1.0,[],0.0,2013-08-29,1.0,1.14,1.47,1


In [18]:
# Define features and target
x = df.drop(columns=['date', 'purchase_owner', 'sale_owner', 'sale_speculation', 'purchase_speculation', 'price_change', 'greater_than'])
y = df['greater_than']

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

tickers = x_test['ticker']
x_train = x_train.drop(columns=['ticker'])
x_test = x_test.drop(columns=['ticker'])


x_train.head()


,purchase_volume,purchase_count,purchase_count_individual,purchase_days_ago,purchase_confidence,sale_volume,sale_count,sale_count_individual,sale_days_ago,sale_confidence,volume_net,score
13634,1.0,1,1,93.0,1.154507,5.0,4,3,65.25,-0.100667,-4.0,1.10
8992,0.0,0,0,-1.0,0.000000,2.0,2,2,105.00,-0.123709,-2.0,0.07
8603,2.0,2,1,120.0,1.399048,0.0,0,0,-1.00,0.000000,2.0,6.04
10144,1.0,1,1,79.0,1.377273,0.0,0,0,-1.00,0.000000,1.0,1.09
3468,0.0,0,0,-1.0,0.000000,1.0,1,1,66.00,0.144062,-1.0,-0.14


In [19]:
# Initialize and train the XGBoost classifier
xgb_classifier = XGBClassifier(random_state=42)
xgb_classifier.fit(x_train, y_train)

# Make predictions
y_pred = xgb_classifier.predict(x_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:')
print(classification_report(y_test, y_pred))
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.70
Classification Report:
              precision    recall  f1-score   support

           0       0.72      0.88      0.79      1857
           1       0.62      0.36      0.46      1004

    accuracy                           0.70      2861
   macro avg       0.67      0.62      0.63      2861
weighted avg       0.69      0.70      0.68      2861

Confusion Matrix:
[[1635  222]
 [ 638  366]]
